##### Config

In [1]:
%matplotlib inline

In [2]:
%config InlineBackend.figure_format = "retina"

In [3]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

# Disable annoying font warnings
matplotlib.font_manager._log.setLevel(50)

# Disable theano deprecation warnings
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=UserWarning, module='theano')

# Style
plt.style.use('default')
plt.rcParams['savefig.dpi'] = 100
plt.rcParams['figure.dpi'] = 100
plt.rcParams['figure.figsize'] = (12, 4)
plt.rcParams['font.size'] = 14
plt.rcParams['text.usetex'] = False
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = ['Liberation Sans']
plt.rcParams['font.cursive'] = ['Liberation Sans']
plt.rcParams['mathtext.fontset'] = 'cm'
plt.rcParams['mathtext.fallback_to_cm'] = True

In [4]:
del matplotlib; del plt; del warnings

##### Main

In [5]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
from scipy.special import factorial
from utils import g, norm_cov
import george
from george.kernels import CosineKernel
from scipy.linalg import toeplitz
from corner import corner

In [371]:
np.random.seed(0)

# GP mean
mu = 0.75

# Dimension of the problem
K = 3

# Order we will go up to
N = 10

# Number of samples in numerical estimate
M = 100000

# Random covariance and its Cholesky decomp.
L = 0.1 * np.tril(0.25 * np.random.randn(K, K) + np.eye(K))
Sig = L @ L.T

# The mean of all elements in the covariance matrix
barSig = np.mean(Sig)

# Ones vector & matrix
j = np.ones((K, 1))
J = np.ones((K, K))

Direct sampling:

In [372]:
u = np.random.randn(K, M)
x = mu + L @ u
xnorm = x / np.mean(x, axis=0).reshape(1, -1)
np.cov(xnorm)

array([[ 0.01518063, -0.00914032, -0.00604031],
       [-0.00914032,  0.02104074, -0.01190042],
       [-0.00604031, -0.01190042,  0.01794073]])

Numerical, using P, Q, R:

In [373]:
P = np.empty((M, K, K))
Q = np.empty((M, K, K))
R = np.empty((M, K, K))

for m in tqdm(range(M)):
    u = np.random.randn(K, 1)
    z = 1 / (mu * K) * j.T @ L @ u
    P[m] = (z ** 2 * J) / (1 + z) ** 2
    Q[m] = (z * L @ u @ j.T) / (1 + z) ** 2
    R[m] = (L @ u @ u.T @ L.T) / (1 + z) ** 2
    
P = np.mean(P, axis=0)
Q = np.mean(Q, axis=0)
R = np.mean(R, axis=0)

P - (Q + Q.T) / mu + R / mu ** 2

array([[ 0.01508355, -0.00904596, -0.00603759],
       [-0.00904596,  0.02100469, -0.01195873],
       [-0.00603759, -0.01195873,  0.01799632]])

Series solution:

In [374]:
def norm_cov(mu, Sig, N=20):
    K = Sig.shape[0]
    J = np.ones((K, K))
    barSig = np.mean(Sig)

    # Powers of S
    S = (Sig @ J) / K ** 2
    Spow = [np.eye(K)]
    for n in range((N + 2) // 2):
        Spow.append(Spow[-1] @ S)

    norm_cov = np.zeros_like(Sig)
    for n in range(0, N + 1, 2):

        EP = (n + 1) * g(n) * barSig ** ((n + 2) // 2) * J
        P = (-1) ** n * (n + 1) * EP
        
        EQ = (n + 1) * K * g(n) * Spow[(n + 2) // 2]
        Q = (-1) ** n * (n + 1) * EQ
        
        ER = n * g(n) * Spow[n // 2] @ Sig + g(n) * barSig ** (n // 2) * Sig
        R = (-1) ** n * (n + 1) * ER
        
        norm_cov += (P - (Q + Q.T) + R) / (mu ** (n + 2))

    return norm_cov

norm_cov(mu, Sig)

array([[ 0.0153166 , -0.00917094, -0.00614566],
       [-0.00917094,  0.02107963, -0.01190869],
       [-0.00614566, -0.01190869,  0.01805435]])